# Milestone 1

In [3]:
import requests
import json
import os
from urllib.request import urlretrieve
import zipfile
import pandas as pd
import glob
import re

DataTransformerRegistry.enable('data_server')

## Downloading Data

In [29]:
# Necessary metadata
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data/"

In [30]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
file_to_dl = files[3]
os.makedirs(output_directory, exist_ok=True)
urlretrieve(file_to_dl["download_url"], output_directory + file_to_dl["name"])

('../data/data.zip', <http.client.HTTPMessage at 0x7ff73001c760>)

In [5]:
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

## Combining Data CSVs

In [6]:
%%time
use_cols = [
    "time",
    "lat_min",
    "lat_max",
    "lon_min",
    "lon_max",
    "rain (mm/day)"
]
files = glob.glob("../data/*.csv")
files.remove("../data/observed_daily_rainfall_SYD.csv")
df = pd.concat(
    (
        pd.read_csv(file, index_col=0, usecols=use_cols).assign(
            model=re.findall(r"../data/(.*)_daily", file)[0]
        )
        for file in files
    )
)
df.to_csv("../data/combined_data.csv")

CPU times: user 5min 42s, sys: 8.74 s, total: 5min 51s
Wall time: 5min 53s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Daniel King |  macOS 12.0.1    |16 GB|Apple M1 2020|  Yes | 5min 53s   |
| Member 2    |                  |     |           |        |            |
| Member 3    |                  |     |           |        |            |
| Member 4    |                  |     |           |        |            |

##  Load the combined CSV to memory and perform a simple EDA

In [22]:
df = pd.read_csv("../data/combined_data.csv", parse_dates=True, index_col=0)

In [20]:
df

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
...,...,...,...,...,...,...,...
62467838,2014-12-27 12:00:00,-30.157068,-29.214660,153.1250,154.3750,6.689683e+00,SAM0-UNICON
62467839,2014-12-28 12:00:00,-30.157068,-29.214660,153.1250,154.3750,7.862555e+00,SAM0-UNICON
62467840,2014-12-29 12:00:00,-30.157068,-29.214660,153.1250,154.3750,1.000503e+01,SAM0-UNICON
62467841,2014-12-30 12:00:00,-30.157068,-29.214660,153.1250,154.3750,8.541592e+00,SAM0-UNICON


Using altair, I ran into the problem of altair not being able to handle to much data at once. So I set the altair data transformer to data server to fix this problem. After creating the plot again, nothing appeared except for two axes, and a "tornado.general" warning. Changing the dtype of the numeric variables from float64 to float32 or float16 did not help.

In [34]:
%%time
print(df.info())
print(df["model"].value_counts())
df["rain (mm/day)"].describe()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62467843 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float64
 1   lat_max        float64
 2   lon_min        float64
 3   lon_max        float64
 4   rain (mm/day)  float64
 5   model          object 
dtypes: float64(5), object(1)
memory usage: 3.3+ GB
None
MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-L

count    5.924854e+07
mean     1.901170e+00
std      5.585735e+00
min     -3.807373e-12
25%      3.838413e-06
50%      6.154947e-02
75%      1.020918e+00
max      4.329395e+02
Name: rain (mm/day), dtype: float64

The combined data takes around 3.3 GB of space. Running the `.info` method, obtaining the model `.values_counts`, and getting the summary statistics of the rainfall using the `.describe` method takes the following amount of time for each group member:

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Daniel King |  macOS 12.0.1    |16 GB|Apple M1 2020|  Yes | 6.57 s     |
| Member 2    |                  |     |           |        |            |
| Member 3    |                  |     |           |        |            |
| Member 4    |                  |     |           |        |            |

### Changing the dtype to float32

Here we reduce the data type of the numeric variables in the data set to `float32` to reduce the memory space that they take up. We tried using `float16`, which resulted in a much lower sys time than float64 but for some odd reason increased the user time.

In [39]:
df_32 = df.astype(
    {
        "lat_min": "float32",
        "lat_max": "float32",
        "lon_min": "float32",
        "lon_max": "float32",
        "rain (mm/day)": "float32",
    }
)

In [41]:
%%time
print(df_32.info())
print(df_32["model"].value_counts())
df_32["rain (mm/day)"].describe()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62467843 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float32
 1   lat_max        float32
 2   lon_min        float32
 3   lon_max        float32
 4   rain (mm/day)  float32
 5   model          object 
dtypes: float32(5), object(1)
memory usage: 2.1+ GB
None
MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-L

count    5.924854e+07
mean     1.901173e+00
std      5.585735e+00
min     -3.807373e-12
25%      3.838413e-06
50%      6.154947e-02
75%      1.020918e+00
max      4.329395e+02
Name: rain (mm/day), dtype: float64

After converting all of the numeric data types to `float16`, the combined data takes around 1.5 GB of space. Running the `.info` method, obtaining the model `.value_counts`, and getting the summary statistics of the rainfall using the `.describe` method takes the following amount of time for each group member:

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Daniel King |  macOS 12.0.1    |16 GB|Apple M1 2020|  Yes | 5.29 s     |
| Member 2    |                  |     |           |        |            |
| Member 3    |                  |     |           |        |            |
| Member 4    |                  |     |           |        |            |

### Only loading necessary columns

Here we reduce the space the data set takes up in memory by only loading the model, time, and rainfall variables.

In [42]:
df_red = pd.read_csv(
    "../data/combined_data.csv",
    parse_dates=True,
    usecols=["time", "rain (mm/day)", "model"],
    index_col=0
)

In [43]:
%%time
print(df_red.info())
print(df_red["model"].value_counts())
df_red["rain (mm/day)"].describe()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62467843 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 2 columns):
 #   Column         Dtype  
---  ------         -----  
 0   rain (mm/day)  float64
 1   model          object 
dtypes: float64(1), object(1)
memory usage: 1.4+ GB
None
MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM   

count    5.924854e+07
mean     1.901170e+00
std      5.585735e+00
min     -3.807373e-12
25%      3.838413e-06
50%      6.154947e-02
75%      1.020918e+00
max      4.329395e+02
Name: rain (mm/day), dtype: float64

The memory the data set took up after only loading in the time, rainfall, and model variables was around 1.4 GB. The wall time actually increased slightly in comparison to the `float32` dataframe  even though the memory used is smaller. This is likely the case because running the describe method on the rainfall column likely negates the memory savings obtained from reducing the columns loaded in.

## Performing a Simple EDA in R

The parquet and feather methods serialize from python to an intermediate file first which is then deserialized into R. However, the pandas and arrow exchange methods do the serialization and deserialization from python to R while skipping the intermediate file, saving computation time. Because we are transferring data from python to R within the same notebook it does not seem necessary to get the benefits of storing data to disk that comes with the parquet and feather methods. Arrow exchange was chosen over pandas exchange because the arrow in-memory files are a unified cross-platform format which means the serialization/deserialization process is much faster than if we used pandas exchange as pandas and R store the in-memory files in different formats. Arrow is also integrated into dplyr and offers many memory-optimized computation methods, which means the arrow format which was transferred over to R will perform much faster computations.

In [1]:
%reset -f

In [2]:
%load_ext rpy2.ipython

In [3]:
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

In [4]:
pydata = ds.dataset("../data/combined_data.csv", format="csv")
pytable = pydata.to_table()
rtable = pyra.converter.py2rpy(pytable)

In [5]:
%%R -i rtable
suppressMessages(library(dplyr))
suppressMessages(library(arrow))
rdata <- rtable

In [6]:
%%R

rtibble <- rtable |>
    collect()
rtibble

# A tibble: 62,467,843 × 7
   time                lat_min lat_max lon_min lon_max `rain (mm/day)` model    
   <dttm>                <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>    
 1 1889-01-01 04:00:00   -35.4   -33.6    142.    143.        4.24e-13 MPI-ESM-…
 2 1889-01-02 04:00:00   -35.4   -33.6    142.    143.        4.22e-13 MPI-ESM-…
 3 1889-01-03 04:00:00   -35.4   -33.6    142.    143.        4.50e-13 MPI-ESM-…
 4 1889-01-04 04:00:00   -35.4   -33.6    142.    143.        4.25e-13 MPI-ESM-…
 5 1889-01-05 04:00:00   -35.4   -33.6    142.    143.        4.27e-13 MPI-ESM-…
 6 1889-01-06 04:00:00   -35.4   -33.6    142.    143.        4.20e-13 MPI-ESM-…
 7 1889-01-07 04:00:00   -35.4   -33.6    142.    143.        4.19e-13 MPI-ESM-…
 8 1889-01-08 04:00:00   -35.4   -33.6    142.    143.        4.56e-13 MPI-ESM-…
 9 1889-01-09 04:00:00   -35.4   -33.6    142.    143.        2.53e+ 0 MPI-ESM-…
10 1889-01-10 04:00:00   -35.4   -33.6    142.    143.        4.12e- 2 MPI-ESM-…
#

In [7]:
%%R

rtable |> 
    count(model) |> 
    collect() |> 
    print(n=27)

# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 MPI-ESM-1-2-HAM   966420
 2 AWI-ESM-1-1-LR    966420
 3 NorESM2-LM        919800
 4 ACCESS-CM2       1932840
 5 FGOALS-f3-L      3219300
 6 CMCC-CM2-HR4     3541230
 7 MRI-ESM2-0       3037320
 8 GFDL-CM4         3219300
 9 BCC-CSM2-MR      3035340
10 EC-Earth3-Veg-LR 3037320
11 CMCC-ESM2        3541230
12 NESM3             966420
13 MPI-ESM1-2-LR     966420
14 ACCESS-ESM1-5    1610700
15 FGOALS-g3        1287720
16 INM-CM4-8        1609650
17 MPI-ESM1-2-HR    5154240
18 TaiESM1          3541230
19 NorESM2-MM       3541230
20 CMCC-CM2-SR5     3541230
21 KIOST-ESM        1287720
22 INM-CM5-0        1609650
23 MIROC6           2070900
24 BCC-ESM1          551880
25 GFDL-ESM4        3219300
26 CanESM5           551880
27 SAM0-UNICON      3541153


In [ ]:
%%R

rtable |>
    rename(rain = "rain (mm/day)") |> 
    select(rain) |> 
    summarize(
        count = n(),
        mean = mean(rain, na.rm = TRUE),
        std = sd(rain, na.rm = TRUE)
    ) |> 
    collect()